## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Прежде чем приступить к анализу, нам необходимо обработать пропущенные значения, выявить и заменить неверные типы данных, найти в таблице дубликаты и удалить их, а также выяснить, какие данные можно объединить в категории. 
Затем мы должны прийти к выводу, как на погашение кредита в срок влияют такие факторы, как наличие детей, семейное положение и уровень дохода заемщиков, и на какие цели они оформляют кредиты. 

### 1. Открытие данных <a id="start"></a>

Для начала работы импортируем таблицу и посмотрим на первые 20 строк, чтобы ознакомиться с данными. Сохраним таблицу под именем *data*. 
Также импортируем библиотеку *PyMystem* и контейнер *Counter* из модуля *Collections*, они пригодятся нам в будущем:

In [4]:
import pandas as pd
from pymystem3 import Mystem
m = Mystem()

from collections import Counter
import warnings 
warnings.filterwarnings('ignore')

In [5]:
data = pd.read_csv('/datasets/data.csv')
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Рассмотрим каждый столбец по отдельности:  
- *children* — количество детей в семье;
- *days_employed* — общий трудовой стаж в днях;
- *dob_years* — возраст клиента в годах;
- *education* — уровень образования клиента;
- *education_id* — идентификатор уровня образования;
- *family_status* — семейное положение;
- *family_status_id* — идентификатор семейного положения;
- *gender* — пол клиента;
- *income_type* — тип занятости;
- *debt* — имел ли задолженность по возврату кредитов;
- *total_income* — ежемесячный доход;
- *purpose* — цель получения кредита.

Получим общую информацию о таблице:

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Всего в таблице 12 столбцов. В них встречаются три типа данных: строки (*object*), вещественные (*float64*) и целые (*int64*) числа.

### Вывод 

Ознакомившись с информацией в таблице, мы пришли к выводу, что не все столбцы пригодятся нам в работе. Так, для ответов на ключевые вопросы исследования нам скорее всего не понадобятся данные о возрасте и поле клиентов и уровне их образования.

Также по первым строкам таблицы можно предположить, что женатые пары чаще берут кредит, связанный с недвижимостью, а клиенты, состощие в гражданском браке, заинтересованы в заемах для проведения свадьбы. Также среди клиентов банка гораздо больше людей среднего возраста, чем 20-летних. 

### 2. Предобработка данных <a id="preprocessing"></a>

### 2.1 Обработка пропущенных значений <a id="null"></a>

Выясним количество пропусков во всей таблице методом *isnull()* + *sum()*:

In [7]:
data.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В столбцах *days_employed* (общий трудовой стаж) и *total_income* (ежемесячный доход) есть пропущенные значения. 

Также в столбце *days_employed* встречаются аномально большие значения: например, у 53-летнего клиента указан стаж в 340 266 дней (или 932 года). Возможно, здесь ошибочно стаж записали в часах, а не в днях. 
Для конечных целей этого исследования нам не понадобятся данные из столбца с трудовым стажем, поэтому смысла переводить часы в дни нет. Но нужно сообщить разработчикам об ошибках в этих данных. 
А **пропуски в *days_employed* можно заменить на медианное значение**.

Также при просмотре столбца *days_employed* в глаза бросаются отрицательные значения. Можем предположить, что они стали отрицательными по ошибке. 
Возьмем для примера первую строчку: клиенту 42 года, трудовой стаж — -8437 дней. Если 8437 дней мы поделим на 365 дней, получим 23 года стажа. Что будет значить, что данный клиент начал работать в 19 лет. 
Вероятно, эти данные действительно стали отрицательными из-за ошибки (об этом тоже нужно сообщить разработчикам). 

Прежде чем заполнять пропуски, переведем отрицательные значения в положительные, использовав метод *apply()*:

In [8]:
data['days_employed'] = data['days_employed'].apply(abs)

#проверим результат, посчитав общее количество отрицательных значений в столбце
data.loc[data['days_employed'] < 0, 'days_employed'].count()

0

Теперь пропуски в *days_employed* можно заменить на медианное значение. Но у людей разных возрастов и разных профессий общий стаж будет отличаться: у 60-летнего пенсионера он гораздо больше, чем у 22-летнего студента. Поэтому медианное значение трудового стажа стоит определять исходя из типа занятости клиента.
Выясним медианное значение трудового стажа по типу занятости:

In [9]:
data.groupby('income_type')['days_employed'].median()

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

Оказалось, что медианный стаж безработных превышает значение стажа пенсионеров. Вероятно, это связано с ошибкой в данных, о которой мы писали ранее (в некоторых строках стаж скорее всего указан в часах).
    
Потому как для нашего исследования нам не понадобятся данные о трудовом стаже, можно заменить пропуски медианой.    
С помощью метода *transform* заменим пропуски в столбце *days_employed*, сгруппировав данные по типу занятости (столбец *income_type*)

In [10]:
#применим метод transform к столбцу days_employed
data['days_employed'] = data['days_employed'].fillna(data.groupby('income_type')['days_employed'].transform('median'))

#проверим, остались ли в этом столбце пропуски:
data['days_employed'].isnull().sum() 

0

В столбце *total_income* также есть NaN-значения. Но в данном случае заполнить все пропуски медианным значеним нельзя. Было бы правильней выяснить медиану **для каждого из типов занятости** и заполнить пропуски именно этим значением.

Выясним, какие типы занятости занесены в нашу таблицу методом *unique()*

In [11]:
data['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

Проверим, как распределились пропуски в столбце с уровнем дохода по типу занятости клиентов, и есть ли в этом закономерность.

In [12]:
#посчитаем, сколько в строках с типом занятости 
#пропущено доходов методом value_counts()
data['income_type'][data['total_income'].isnull()].value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

Больше всего данных нет у сотрудников, компаньонов и пенсионеров. В меньшей степени — у госслужащих и предпринимателей. Получается, даже у категории "безработные" есть сведения о доходах.

Сгруппируем данные по типу занятости и определим медианный доход для каждого из них:

In [13]:
data.groupby('income_type')['total_income'].median()

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

Мы вычислили медианные заработки, а также выяснили, что никаких артефактов или дублирующихся значений в столбце с видом занятости нет, эти данные готовы к анализу. 

Теперь мы можем заменить пропущенные значения медианным для каждого из типов занятости. Доход составит:
* для сотрудника — 142594.396847
* для компаньона — 172357.950966
* для пенсионера — 118514.486412
* для госслужащего — 150447.935283
* для предпринимателя — 499163.144947

In [14]:
#заменим пропущенные значения для каждой категории занятости медианой с помощью функции trasnform()
data['total_income'] = data['total_income'].fillna(data.groupby('income_type')['total_income'].transform('median'))

#проверим функцией try-except, остались ли пропуски в этом столбце
try:
    #вызовем подсчет пропущенных значений, используя count()
    print(data.loc[data['total_income'] == 'NaN', 'total_income'].count())
except:
    print('Проверьте ваш код')

0


Проверим, остались ли во всей таблице *data* пропущенные значения:

In [15]:
data.isnull().sum()
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Вывод

На этапе обработки пропусков обнаружили не только NaN-данные, но и отрицательные и аномально большие значения в столбце с трудовым стажем. Об этих ошибках лучше сообщить разработчикам. Пропуски в трудовом стаже заменили на медианное значение всего столбца. В столбце с ежемесячным доходом мы заполнили пустые строки в соответствии с медианой для каждого из типов занятости. 

### 2.2 Замена типа данных <a id="type"></a>

Одна из наших задач — выявить зависимость между уровнем дохода и возвратом кредита в срок. Сейчас столбец с уровенем дохода (*total_income*) имеет тип данных *float64*. Чтобы было удобнее работать с числовыми значениями, лучше их округлить — то есть заменить на тип данных *int64*. Выполним задачу при помощи метода *astype()*:

In [16]:
data['total_income'] = data['total_income'].astype('int')
#проверим по первым пяти строкам таблицы
data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


### Вывод 
Значения в столбце с ежемесячным доходом стали целочисленными. 

Этот же метод можно было бы применить к столбцу с трудовым стажем, но для нашего исследования эти данные не понадобятся, поэтому заменять в нем тип *float64* не имеет смысла. 
В остальных столбцах замена данных не требуется.

### 2.3 Обработка дубликатов <a id="duplicates"></a>

Продолжим обработку данных интересующих нас столбцов — найдем в них артефакты и дубликаты.

Сразу обращаем внимание на наличие дубликатов в столбце с информацией об образовании клиентов: различается регистр в слове "средний" — оно записано и со строчной, и прописной буквы. Возможно, это связано с тем, что эту графу клиенты заполняют в анкете вручную, самостоятельно, поэтому здесь нет единообразия в оформлении. 

Методом *str.lower()* приведем все слова в этом столбце к нижнему регистру.

In [17]:
data['education'] = data['education'].str.lower()

#проверим, работает ли код, просмотрев уникальные значения методом unique()
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

Теперь все данные об образовании приведены к единообразию.
Далее проанализируем столбец с количеством детей методом *value_counts()* на наличие артефактов. 

In [18]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

В таблице обнаружили странные значения: **-1** и **20**. Скорее всего, это опечатки. Можно предположить, что вместо **-1** подразумевался один ребенок. Вместо **20** наверняка хотели указать двоих детей, потому как маловероятным кажется разрыв между двумя максимальными значениями этого столбца: 5 и 20 детей. 
Медотом *replace()* заменим **-1** на **1**, а **20** — на **2**:

In [19]:
data['children'] = data['children'].replace(-1, 1)
data['children'] = data['children'].replace(20, 2)
#проверим, сработала ли замена, просмотрев уникальные значения столбце children
data['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Теперь никаких аномалий в этом столбце нет.
Тем же методом проверим столбец с данными о семейном положении клиентов.

In [20]:
data['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

Здесь никаких проблемных значений нет. 

### Вывод

Среди дубликатов нам встретились строки с информацией об образовании, записанные в разных регистрах. Мы привели к единообразию эти данные. 
Помимо этого, мы столкнулись со необычными (и, вероятно, ошибочными) данными в графе "количество детей" — о них также стоит сообщить разработчикам. Для нашего исследования отрицательное число (-1) мы заменили на положительное, а слишком большое количество (20 детей) заменили на 2. 

### 2.4 Лемматизация <a id="lemma"></a>

Первых строк таблицы достаточно, чтобы понять, что последний столбец с целями кредита заполнялся клиентами банка самостоятельно. Так например, есть аналогичные по смыслу значения "покупка жилья", "операции с жильем" и "покупка жилья для семьи". Очевидно, что в таблице имеются и другие варианты. При помощи метода *unique()* узнаем уникальные значения столбца "цели кредита":

In [21]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Теперь мы можем приступить к лемматизации — приведению слов к их словарной (начальной) форме. 

In [22]:
#в переменной text зададим уникальные значения столбца purpose
#при этом методом join() преобразуем список уникальных значений в строку 
text = ' '.join(data['purpose'].unique())

#создадим переменную lemmas - список леммантизированных слов
#в качестве аргумента укажем нашу переменную text 
lemmas = m.lemmatize(text)

#при помощи контейнера Counter посчитаем количество упоминаний леммантизированных слов
Counter(lemmas)

Counter({'покупка': 10,
         ' ': 96,
         'жилье': 7,
         'приобретение': 1,
         'автомобиль': 9,
         'дополнительный': 2,
         'образование': 9,
         'сыграть': 1,
         'свадьба': 3,
         'операция': 4,
         'с': 5,
         'на': 4,
         'проведение': 1,
         'для': 2,
         'семья': 1,
         'недвижимость': 10,
         'коммерческий': 2,
         'жилой': 2,
         'строительство': 3,
         'собственный': 1,
         'подержать': 2,
         'свой': 4,
         'со': 1,
         'заниматься': 2,
         'сделка': 2,
         'получение': 3,
         'высокий': 3,
         'профильный': 1,
         'сдача': 1,
         'ремонт': 1,
         '\n': 1})

Выявим основные леммы: 
* жилье,
* автомобиль,
* свадьба, 
* недвижимость, 
* строительство, 
* сделка,
* сдача,
* ремонт.

В полученном списке можно выделить следующие цели кредита:
* операции с жильем и ремонт;
* покупка автомобиля;
* проведение свадьбы;
* получение образования.

Теперь мы можем задать собственные категории. Напишем функцию, которая будет заменять имеющиеся значения на четыре категории: *недвижимость*, *автомобиль*, *образование*, *свадьба*. "Ремонт" также отнесем к *недвижимости*, потому как эта цель кредита встречается всего один раз. 

In [23]:
#создаем функцию purpose_new
def purpose_new(lemmas):
    #если в нашем списке леммантизированных слов lemmas встретится "жилье",
    #то нужно вернуть значение "недвижимость"
    if 'жилье' in lemmas:
        return 'недвижимость'
    #продолжим функцию для всех категорий цели кредита
    if 'недвижимость' in lemmas:
        return 'недвижимость'
    if 'автомобиль'in lemmas:
        return 'автомобиль'
    if 'образование'in lemmas:
        return 'образование'
    if 'свадьба'in lemmas:
        return 'свадьба'
    return 'другое'

#создадим отдельный столбец для наших лемм - lemmas
data['lemmas'] = data['purpose'].apply(m.lemmatize)
#создадим новый столбец purpose_group, применив нашу функцию purpose_new  
data['purpose_group'] = data['lemmas'].apply(purpose_new) 
#проверим и посчитаем уникальные значения нового столбца purpose_group
data['purpose_group'].value_counts()

недвижимость    10840
автомобиль       4315
образование      4022
свадьба          2348
Name: purpose_group, dtype: int64

### Вывод

Чаще всего клиенты оформляют заем на операции с жилой и коммерческой недвижимостью. На втором месте по популярности — кредиты на покупку автомобиля и получение образования. Еще в два раза реже люди используют заемные средства для проведения свадеб. 

### 2.5 Категоризация данных <a id="category"></a>

Для исследования нам необходимо категоризировать данные по ежемесячному доходу клиентов банка. Используем метод *qcut()*, чтобы распределить значения столбца *total_income* по четырем категориям. При этом количество значений в каждой из категорий будет примерно одинаковым. 

In [24]:
#создадим новый столбец categories_income с нашими категориями доходов
data['categories_income'] = pd.qcut(data['total_income'], 4)
data['categories_income'].value_counts()

(107798.0, 142594.0]     5523
(20666.999, 107798.0]    5382
(195549.0, 2265604.0]    5381
(142594.0, 195549.0]     5239
Name: categories_income, dtype: int64

Определились доходы клиентов по четырем категориям:
* низкий доход — до 107 798 рублей;
* средний доход — от 107 798 до 142 594 рублей;
* выше среднего — от 142 594 до 195 549 рублей;
* высокий доход — более 195 549.

Переименуем строки в столбце *categories_income* для удобства восприятия

In [25]:
#создадим функцию income_category, определяющую уровень дохода:
def income_category(income):
    #распределим доходы по четырем категориям
    if income < 107798:
        return 'низкий'
    if income < 142594:
        return 'средний'
    if income < 195549:
        return 'выше среднего'
    if income >= 195549:
        return 'высокий'
    return 'другое'

#применим функцию income_category к столбцу categories_income
data['categories_income'] = data['total_income'].apply(income_category)
#проверим, как распределились доходы клиентов
data['categories_income'].value_counts()

выше среднего    6343
высокий          5382
низкий           5381
средний          4419
Name: categories_income, dtype: int64

### Вывод

Чаще остальных за кредитом в банк обращаются люди с доходом выше среднего. У банка примерно одинаковое число клиентов с высоким и низким заработком. Реже заемщиками становятся люди со средним доходом. 

### 3. Выводы по поставленным вопросам <a id="#answers"></a> 

### 3.1 Влияет ли наличие детей на возврат кредита в срок? <a id="kids"></a>

Для ответа на этот вопрос необходимо создать сводную таблицу с информацией о количестве детей в семьях клиентов и наличием/отсутствием у них задолженности по кредиту. 

In [26]:
#создаем таблицу kids_ratio 
kids_ratio = data.pivot_table(index=['children'], columns='debt', values='total_income', aggfunc='count')
#выведем нашу таблицу на экран
kids_ratio

debt,0,1
children,,
0,13086.0,1063.0
1,4420.0,445.0
2,1929.0,202.0
3,303.0,27.0
4,37.0,4.0
5,9.0,NaN


В столбце *children* записано количество детей, в строке *debt* **0** означает, что задолженности нет, **1** — у клиента есть долг. 

Подавляющее большинство клиентов возвращают кредиты в срок. Но прежде чем перейти к выявлению доли должников, нам необходимо удалить пропущенные значения в получившейся таблице. Заменим их на нули методом *fillna()*:

In [27]:
kids_ratio[1] = kids_ratio[1].fillna(0)

#для удобства переименуем столбцы: 0 - without_debt, 1 - with_debt
kids_ratio.set_axis(['without_debt', 'with_debt'], axis = 'columns', inplace=True)
kids_ratio

,without_debt,with_debt
children,,
0,13086.0,1063.0
1,4420.0,445.0
2,1929.0,202.0
3,303.0,27.0
4,37.0,4.0
5,9.0,0.0


Теперь можно перейти в вычислению соотношения между наличием долга по кредиту и количеству детей в семье. Выясним процентное соотношение должников к общему числу бездетных клиентов или тех, кто имеет одного и более детей.  

In [28]:
#создадим новый столбец
kids_ratio['with_debt, %'] = kids_ratio['with_debt'] / (kids_ratio['with_debt'] + kids_ratio['without_debt']) * 100
kids_ratio

,without_debt,with_debt,"with_debt, %"
children,,,
0,13086.0,1063.0,7.512898
1,4420.0,445.0,9.146968
2,1929.0,202.0,9.479118
3,303.0,27.0,8.181818
4,37.0,4.0,9.756098
5,9.0,0.0,0.000000


### Вывод

Похоже, что бездетные клиенты чаще оплачивают кредиты в срок. Заемщики с тремя детьми также ответственно подходят к выплате долга. Чуть хуже обстоят дела у должников с 1 или 2 детьми, их примерно одинаковое количество.
При этом клиенты с 5 детьми не задерживают оплату вовсе, а семьи, в которых 4 ребенка, имеют самый высокий процент среди должников. Но общее количество таких семей слишком мало, чтобы делать из этого какие-либо выводы. 

Поэтому в нашем исследовании можно сделать вывод, что наличие или отсутствие у клиента детей не влияет на то, задерживает ли он выплаты по кредиту.

### 3.2 Влияет ли семейное положение на возврат кредита в срок?<a id="family"></a>

Нам снова необходимо создать сводную таблицу, на этот раз сравнивать должников будем по их семейному положению.

In [29]:
#создадим сводную таблицу family_ratio
family_ratio = data.pivot_table(index = 'family_status', columns = 'debt', values = 'total_income', aggfunc = ('count'))

#переименуем столбцы, исходя из наличия/отстутствия задолженности
family_ratio.set_axis(['without_debt', 'with_debt'], axis = 'columns', inplace = True)
family_ratio

,without_debt,with_debt
family_status,,
Не женат / не замужем,2539,274
в разводе,1110,85
вдовец / вдова,897,63
гражданский брак,3789,388
женат / замужем,11449,931


В получившейся таблице нет пропущенных значений, поэтому можно сразу приступать к выявлению доли клиентов, задерживающих выплаты по кредиту:

In [30]:
#новый столбец with_debt, % - с процентным показателем должников
family_ratio['with_debt, %'] = family_ratio['with_debt'] / (family_ratio['with_debt'] + family_ratio['without_debt'])  * 100
family_ratio

,without_debt,with_debt,"with_debt, %"
family_status,,,
Не женат / не замужем,2539,274,9.740491
в разводе,1110,85,7.112971
вдовец / вдова,897,63,6.562500
гражданский брак,3789,388,9.288963
женат / замужем,11449,931,7.520194


### Вывод

Вдовцы, вдовы и разведенные клиенты ответственнее других подходят к погашению задолженностей. Состоящие в браке также стараются оплатить кредиты в срок. 
Больше всего должников среди неженатых и живущих в гражданском браке клиентов.
Видимо, опыт ведения семейного бюджета учит финансовой грамотности — таких клиентов реже встретишь в списках должников. 

### 3.3 Влияет ли уровень дохода на возврат кредита в срок? <a id="income"></a>

Теперь нужно выяснить, клиенты с каким уровнем дохода чаще задерживают выплаты по кредитам. Правда ли, что самые ненадежные заемщики — люди с низким заработком?

В этот раз группировать данные мы будем по созданному нами ранее столбцу с категориями доходов *categories_income*:

In [31]:
#создаем сводную таблицу income_ratio 
income_ratio = data.pivot_table(index = 'categories_income', columns = 'debt', values = 'purpose', aggfunc = 'count')

#переименуем столбцы по наличию/отсутствию долгов
income_ratio.set_axis(['without_debt', 'with_debt'], axis = 'columns', inplace = True)
income_ratio

,without_debt,with_debt
categories_income,,
высокий,4996,386
выше среднего,5800,543
низкий,4954,427
средний,4034,385


Вычислим долю должников по уровню их доходов:

In [32]:
income_ratio['with_debt, %'] = income_ratio['with_debt'] / (income_ratio['with_debt'] + income_ratio['without_debt']) * 100
income_ratio

,without_debt,with_debt,"with_debt, %"
categories_income,,,
высокий,4996,386,7.172055
выше среднего,5800,543,8.560618
низкий,4954,427,7.935328
средний,4034,385,8.712378


### Вывод

Как и можно было ожидать, меньше всего должников среди людей с высоким доходом. При этом низкий доход клиента также показывает, что заемные средства в большинстве случаев будут возвращены в срок. Люди со средним заработком и выше среднего чаще других задерживают выплаты по кредитам.  

### 3.4 Как разные цели кредита влияют на его возврат в срок? <a id="purpose"></a>

Определим, есть ли зависимость между целями кредита и долей должников по ним?

Создадим таблицу и сгруппируем ее по четырем целям кредитов: недвижимость, автомобиль, свадьба и образование. 

In [33]:
#создаем таблицу purpose_ratio
purpose_ratio = data.pivot_table(index = 'purpose_group', columns = 'debt', values = 'total_income', aggfunc = 'count')

#переименуем столбцы: "нет долга" и "есть долг"
purpose_ratio.set_axis(['without_debt', 'with_debt'], axis = 'columns', inplace = True)
purpose_ratio

,without_debt,with_debt
purpose_group,,
автомобиль,3912,403
недвижимость,10058,782
образование,3652,370
свадьба,2162,186


Выясним долю должников по четырем целям кредита:

In [34]:
purpose_ratio['with_debt, %'] = purpose_ratio['with_debt'] / (purpose_ratio['with_debt'] + purpose_ratio['without_debt']) * 100
purpose_ratio

,without_debt,with_debt,"with_debt, %"
purpose_group,,,
автомобиль,3912,403,9.339513
недвижимость,10058,782,7.214022
образование,3652,370,9.199403
свадьба,2162,186,7.921635


### Вывод

Самый популярный тип кредита — на приобретение недвижимости — возвращают в срок чаще остальных. Заем на проведение свадьбы также оплачивают вовремя. Больше всего должников среди клиентов, оформивших кредиты на покупку автомобиля и получение образования.

### 4. Общий вывод <a id="general"></a>

В ходе нашего исследования нам нужно было выяснить, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.

Мы выявили, что вдовцы и вдовы, разведенные и женатые клиенты ответственно относятся к погашению кредита. Неженатые и состоящие в гражданском браке чаще попадают в списки должников.

Бездетные клиенты чаще оплачивают кредиты вовремя. При этом среди семей с 5-ю детьми должников нет совсем. Данные по должникам с 1–4 детьми распределились равномерно. Думаем, можно сделать вывод, что количество детей в семье клиента не влияет на погашение кредита в срок. 